In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
import time
import numpy as np
import seaborn as sns
from IPython.display import clear_output, display

In [ ]:
class board:
    wMoved = {'l':False,'r':False, 'K':False}
    bMoved = {'l':False,'r':False, 'K':False}
    files = ['a','b','c','d','e','f','g','h']
    files_r = ['h','g','f','e','d','c','b','a']
    ranks = [1,2,3,4,5,6,7,8]
    ranks_r = [8,7,6,5,4,3,2,1]
    moveDict = {'upx':('up','x'),
               'downx':('down','x'),
               'leftx':('left','x'),
               'rightx':('right','x'),
               'upxleftx':(('up','x'),('left','x'),2),
               'upxrightx':(('up','x'),('right','x'),2),
               'downxleftx':(('down','x'),('left','x'),2),
               'downxrightx':(('down','x'),('right','x'),2),
               'up1':('up',1),
               'down1':('down',1),
               'left1':('left',1),
               'right1':('right',1),
               'up1left1':(('up',1),('left',1),2),
               'up1right1':(('up',1),('right',1),2),
               'down1left1':(('down',1),('left',1),2),
               'down1right1':(('down',1),('right',1),2),
               'up2left':(('up',2),('left',1),2),
               'up2right':(('up',2),('right',1),2),
               'left2up':(('left',2),('up',1),2),
               'left2down':(('left',2),('down',1),2),
               'right2up':(('right',2),('up',1),2),
               'right2down':(('right',2),('down',1),2),
               'down2left':(('down',2),('left',1),2),
               'down2right':(('down',2),('right',1),2),
               'up2':('up',2),
                'down2':('down',2),
               'WpTakeL':(('up',1),('left',1),2),
               'WpTakeR':(('up',1),('right',1),2),
               'BpTakeL':(('down',1),('left',1),2),
               'BpTakeR':(('down',1),('right',1),2),
               'castleR': ('right',2),
               'castleL': ('left',2)}
    
    def __init__(self, read=False, copyFrom=False, copySource=False, imaginary=False):
        #self.calling = []
        #creates a board as a DataFrame or clones an existing board
        if imaginary or copyFrom:
            self.body = copySource.copy()
        else:
            self.body = pd.read_csv(read, index_col=0)
            
    def promote(self, team, square):
        #self.calling.append('promote')
        #global board
        options = ['Q', 'R', 'B', 'N']
        promotion = random.choice(options)
        newPiece = str(team) + promotion
        self.body.at[square[1],square[0]] = newPiece
        return (str(team) + 'P' + str(square[0]) + str(square[1]) + ' promoted to ' + promotion)
        #print(newPiece)
            
    def myMoves(self, teamdict):
        #self.calling.append('myMoves')
        #print('MYMOVES')
        moves = {}
        for piece in teamdict:
            x = teamdict[piece]
            if x[1] != 0:
                moves[piece] = x
        return moves
    
    def recon(self, rsquare):
        #self.calling.append('recon')
        #look at a square and return the piece ID or False
        if not rsquare:
            return False
        if rsquare == '0':
            return False
        #print('recon rsquare: ' + str(rsquare))
        file = rsquare[0]
        rank = rsquare[1]
        square = self.body.at[rank,file]
        #print('recon square: ' + str(square))
        return square
        # returns False or wKe1, bQa2, etc...
        
    def assessSquare(self, square, newSquare, origin):
        #self.calling.append('assessSquare')
        # look at square, new square, and return friend, foe, oob, or empty
        check = self.recon(newSquare)
        if not check:
            #out of bounds
            #print('oob')
            return 'oob'
        elif check == '0':
            #then the square is empty
            #print('empty')
            return 'empty'
        else:
            #the square is either friend or foe
            #print(recon(newSquare))
            #print(recon(square))
            if self.recon(newSquare)[0] is not self.recon(origin)[0]:
                #print('foe')
                return 'foe'
            else:
                #print('friend')
                return 'friend'
            
    def walk(self, square, movement):
        #self.calling.append('walk')
        # return new square after given square and move ('up', 1)
        if not square:
            return False
        file = square[0]
        rank = square[1]
        direction = movement[0]
        steps = movement[1]
        if steps == 'x':
            steps = 1
        #print(steps)
        fileCom = {"up":1, "down":-1, "left":-1, "right":1}
        global newPos
        if direction in ["left", "right"]:
            findex = board.files.index(file)
            nindex = findex + (steps * fileCom[direction])
            if nindex not in range(0,8):
                return False
            newFile = board.files[nindex]
            newPos = (newFile, rank)
        if direction in ["up", "down"]:
            newRank = (rank + (steps * fileCom[direction]))
            if newRank not in range(1,9):
                return False
            newPos = (file, newRank)
        return newPos
    
    def movePiece(self, square, newSquare):
        #self.calling.append('movePiece')
        self.body.at[newSquare[1], newSquare[0]] = self.body.at[square[1], square[0]]
        self.body.at[square[1], square[0]] = '0'
    
    def xCheck(self, x, move, square, imaginary=False):
        #self.calling.append('xCheck')
        origin = square
        xSquares = []
        #look through each move str name, make moves and report if possible
        if 'x' in move:
            #print(move + ': x')
            #need to test further moves in this direction until blocked
            blocked = False
            #xSquares = []
            while not blocked:
                #print('not blocked')
                if len(x) == 3:
                    #move is a two-step
                    move1 = x[0]
                    #print(move1)
                    move2 = x[1]
                    #print(move2)
                    newSquare = self.walk(square, move1)
                    #print(newSquare)
                    newSquare = self.walk(newSquare, move2)
                    #print(newSquare)
                else:
                    #move is a one-step
                    newSquare = self.walk(square, x)
                    #print(newSquare)

                result = self.assessSquare(square, newSquare, origin)
                #print(result)
                if result == 'oob':
                    blocked = True
                    newSquare = False
                elif result == 'empty':
                    blocked = False
                    xSquares.append(newSquare)
                    square = newSquare
                elif result == 'friend':
                    blocked = True
                    newSquare = False
                else:
                    blocked = True
                    xSquares.append(newSquare)
                    square = newSquare
            newSquare = xSquares
            if len(newSquare) == 0:
                newSquare = False

        if 'x' not in move:
            #print(move + ': no x')
            #move only specified number of moves

            #non-castle
            if len(x) == 3:
                #print(x)
                #move is a two-step
                #this is where the horses move
                move1 = x[0]
                #print(move1)
                move2 = x[1]
                #print(move2)
                newSquare = self.walk(square, move1)
                newSquare = self.walk(newSquare, move2)
            else:
                #move is a one-step
                newSquare = self.walk(square, x)
            result = self.assessSquare(square, newSquare, origin)
            if result == 'oob':
                newSquare = False
            elif result == 'friend':
                if not imaginary:
                    newSquare = False
            if 'P' in self.recon(square):
                if newSquare:
                    if not imaginary:
                        if newSquare[0] is not square[0]:
                            #if newSquare is in different file
                            if self.recon(newSquare) == '0':
                                #if newSquare is empty
                                newSquare = False
                        elif newSquare[0] is square[0]:
                            #if newSquare is in same file
                            if self.recon(newSquare) != '0':
                                #if newSquare is empty
                                newSquare = False
                    elif imaginary:
                        if newSquare[0] is square[0]:
                            #same file
                            #if self.recon(newSquare) != '0':
                                #not empty
                            newSquare = False
            if newSquare:
                xSquares.append(newSquare)
        return xSquares
    
    def castle(self, team, side):
        #self.calling.append('castle')
        #global board
        if team == 'w':
            if side == 'l':
                #king is at ('e',1), rook is at ('a',1)
                #king goes to ('c',1) and left rook goes to ('d',1)
                kingfrom = ('e',1)
                kingto = ('c',1)
                rookfrom = ('a',1)
                rookto = ('d',1)
                return 'white queen side castle'

            if side == 'r':
                #king is at ('e',1), rook is at ('h',1)
                #king goes to ('g',1) and right rook goes to ('f',1)
                kingfrom = ('e',1)
                kingto = ('g',1)
                rookfrom = ('h',1)
                rookto = ('f',1)
                return 'white king side castle'

        if team == 'b':
            if side == 'l':
                #king is at ('e',8), rook is at ('a',8)
                #king goes to ('c',8) and left rook goes to ('d',8)
                kingfrom = ('e',8)
                kingto = ('c',8)
                rookfrom = ('a',8)
                rookto = ('d',8)
                return 'black queen side castle'

            if side == 'r':
                #king is at ('e',8), rook is at ('h',8)
                #king goes to ('g',8) and rook goes to ('f',8)
                kingfrom = ('e',8)
                kingto = ('g',8)
                rookfrom = ('h',8)
                rookto = ('f',8)
                return 'black king side castle'
    
        ### move king ###
        self.movePiece(kingfrom, kingto)
        #################
        ### move rook ###
        self.movePiece(rookfrom, rookto)
        #################
        #return self.body
        
    def defaultMoves(self, square, forCastle=False):
        #self.calling.append('defaultMoves')
        moves = []
        #print('DEFAULTMOVES')
        #returns default moveset for a piece
        file = square[0]
        rank = square[1]
        #print('      defaultMoves is calling recon')
        piece = self.recon(square)
        if piece == '0':
            return False
        color = piece[0]
        #print(color)
        pieceType = piece[1]
        #print(pieceType)
        if pieceType == 'P':
            #check pawn team and loc
            if color == 'w':
                moves = ['up1','WpTakeL','WpTakeR']
                if rank == 2:
                    #pawn can move up 1 or 2
                    moves = ['up1','up2','WpTakeL','WpTakeR']
            if color == 'b':
                moves = ['down1','BpTakeL','BpTakeR']
                if rank == 7:
                    #pawn can move down 1 or 2
                    moves = ['down1','down2','BpTakeL','BpTakeR']
        if pieceType == 'R':
            moves = ['upx','downx','leftx','rightx']
        if pieceType == 'N':
            moves = ['up2left','up2right','left2up','left2down','right2up','right2down','down2left','down2right']
        if pieceType == 'B':
            moves = ['upxleftx','upxrightx','downxleftx','downxrightx']
        if pieceType == 'Q':
            moves = ['upx','downx','leftx','rightx','upxleftx','upxrightx','downxleftx','downxrightx']
        if pieceType == 'K':
            moves = ['up1','down1','left1','right1','up1left1','up1right1','down1left1','down1right1']
            if not forCastle:
                #print('      defaultMoves is calling canCastle')
                left = self.canCastle(color, 'l')
                right = self.canCastle(color, 'r')
                if left:
                    moves.append('castleL')
                if right:
                    moves.append('castleR')
        return moves
    
    def moveTester(self, square, imaginary=False, forCastle=False):
        #self.calling.append('moveTester')
        #returns legal moves of piece in square
        legalmoves = {}
        #print('     moveTester is calling recon')
        if self.recon(square) == '0':
            return False
        #print('     moveTester is calling defaultMoves to check ' + str(square))
        moves = self.defaultMoves(square, forCastle=forCastle)
        #print(moves)
        for move in moves:
            x = board.moveDict[move]
            #print(move)
            #print(x)
            newSquare = self.xCheck(x, move, square, imaginary)
            legalmoves[move] = newSquare

        #returns a dict of moves and their resulting squares (or False)
        return legalmoves